In [1]:
import pandas as pd
import json
#to read multiple json files from directory
#import glob
#using the operating system module to access directory
#import os


In [2]:
#1st function, reading json file and converting to one dataframe using pandas
#if we were to read multiple json files from set directory, def jsondir_to_df(json_dir)
def json_to_df(json_file):
    
    #json_dir = '/marjanasarker/src/take_home_test_os'
    #json_pattern = os.path.join(json_dir, '*.json')
    #file_list = glob.glob(json_pattern) 
    #all_df = []
    
    with open(json_file, "r") as f:
        #reading json file
        data = json.loads(f.read())
        
        #flattening json dictionaries into a dataframe
        flatten_payload_1 = pd.json_normalize(data)
        #print(flatten_payload_1)
        
        #looking for whether there is a list value in one of the df columns 
        for i in range(len(flatten_payload_1.iloc[0])):
            
            #if there is a list, we utilize json_normalize for that column 
            if isinstance(flatten_payload_1.iloc[0][i], list):
                column_normalize=flatten_payload_1.columns[i]
                #print(column_normalize)
                
                #splitting column name for record path parameters
                column_normalize_list=column_normalize.split(".")
                #print(column_normalize_list)
                
                #normalizing list of dictionaries data
                column_normalize_data = pd.json_normalize(data, record_path=[[column_normalize_list[0],column_normalize_list[1]]])
                #print(column_normalize_data)
                
                #if contactType data present, we select for contactType for which data is present 
                #new_df = column_normalize_data.dropna(subset=[col for col in column_normalize_data.columns if "address" in col or "phoneNumber" in col], axis=0, how="all")
                
                #dropping the column that was normalized, from root df
                flatten_payload_1.drop(column_normalize, inplace=True, axis=1)
                #print(flatten_payload_1)
                
                #concatenating dataframes
                frames = [flatten_payload_1, column_normalize_data]
                final_payload=pd.concat(frames).drop_duplicates().reset_index(drop=True)
                
                #dropping records with all NaN/Null values
                final_payload.dropna(axis=0, how="all", inplace=True)
                
                #multiple json files
                #all_df.append(final_payload)
                
    #multiple json files
    #combined_dfs = pd.concat(all_df, axis=1)
    #combined_dfs.dropna(axis=0, how="all", inplace=True)
    #return pd.concat(all_df)
    
    return final_payload 
                
                

In [3]:
final_payload = json_to_df("take_home_sample.json")

In [4]:
final_payload

,payloadId,receivedDateTime,subscriber.subscriberId,subscriber.lastName,subscriber.firstName,subscriber.middleName,contactType,preferenceRank,address.line1,address.line2,address.city,address.state,address.zip,phoneNumber.countryCode,phoneNumber.number
1,NaN,NaN,NaN,NaN,NaN,NaN,address,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,phoneNumber,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#2nd function to generate csv files for the different interfaces
def df_to_csv(final_payload):
    
    #creating subscriber interface 
    subscriber_cols = [col for col in final_payload.columns if "subscriber" in col]
    subscriber_csv = final_payload.to_csv('subscriber_interface_v2.csv', columns=subscriber_cols, index=False)
    
    #creating contactInformation interface
    #this part is hard-coded, this will be changed once json format is modified
    contactInfo_cols = ['contactType', 'preferenceRank']
    contactInfo_csv = final_payload.to_csv('contactInfo_interface_v2.csv', columns=contactInfo_cols, index=False)
    
    #creating address interface
    address_cols = [col for col in final_payload.columns if "address" in col]
    address_csv = final_payload.to_csv('address_interface_v2.csv', columns=address_cols, index=False)
    
    #creating phoneNumber interface
    phoneNumber_cols = [col for col in final_payload.columns if "phoneNumber" in col]
    phoneNumber_csv = final_payload.to_csv('phoneNumber_interface_v2.csv', columns=phoneNumber_cols, index=False)
    
    return subscriber_csv, contactInfo_csv, address_csv, phoneNumber_csv
    
    

In [6]:
csv_interfaces = df_to_csv(final_payload)